# Model evaluation - Optimal thresholding on default probas
In this notebook, we use a LogReg trained on 1M SIRETs, evaluated on 250k SIRETs.<br>
We compute $f_{\beta}$ scores, balanced accuracy and select thresholds optimally.

In [ ]:
# Set logging level to INFO
import logging
logging.getLogger().setLevel(logging.INFO)

# Import required libraries and modules
from datetime import datetime
from pathlib import Path
import pickle
import json
from types import ModuleType

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, fbeta_score, average_precision_score
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper

from predictsignauxfaibles.config import IGNORE_NA
from predictsignauxfaibles.data import SFDataset
from predictsignauxfaibles.evaluate import evaluate, make_precision_recall_curve, make_thresholds_from_fbeta, make_thresholds_from_conditions
from predictsignauxfaibles.merge_models import merge_models
from predictsignauxfaibles.pipelines import run_pipeline
from predictsignauxfaibles.utils import assign_flag, load_conf, log_splits_size

In [ ]:
# Mute logs from sklean_pandas
logging.getLogger("sklearn_pandas").setLevel(logging.WARNING)

### Loading predictions from csv, splitting models

In [ ]:
## LOADING MODELS FROM FILES

model_runs = {
    "default": "/home/simon.lebastard/predictsignauxfaibles/predictsignauxfaibles/model_runs/default_20210603-131120",
    "small": "/home/simon.lebastard/predictsignauxfaibles/predictsignauxfaibles/model_runs/small_20210603-141114"
}

## DEFAULT MODEL PIPELINE

default = pd.read_csv(f"{model_runs['default']}/predictions.csv")
default_mapper_unpickled =  pickle.load(
    open( f"{model_runs['default']}/model_comp0.pickle",
        "rb"
        )
)
default_mapper = default_mapper_unpickled[1]
default_model_unpickled = pickle.load(
    open( f"{model_runs['default']}/model_comp1.pickle",
        "rb"
        )
)
default_model = default_model_unpickled[1]

default_pp = Pipeline(
    [("transform_dataframe", default_mapper), ("fit_model", default_model)]
)

## SMALL MODEL PIPELINE

small = pd.read_csv(f"{model_runs['small']}/predictions.csv")
small_mapper_unpickled = pickle.load(
    open( f"{model_runs['small']}/model_comp0.pickle",
        "rb"
        )
)
small_mapper = small_mapper_unpickled[1]
small_model_unpickled = pickle.load(
    open( f"{model_runs['small']}/model_comp1.pickle",
        "rb"
        )
)
small_model = small_model_unpickled[1]

small_pp = Pipeline(
    [("transform_dataframe", small_mapper), ("fit_model", small_model)]
)

A couple of quicks checks/diagnoses:

In [ ]:
default.shape

In [ ]:
small.shape

## Choosing model to select thresholds on

In [ ]:
model_name = "small"

if model_name == "default":
    conf = load_conf(model_name="default")
    pp = default_pp
    mapper = default_mapper
    model = default_model
if model_name == "small":
    conf = load_conf(model_name="small")
    pp = small_pp
    mapper = small_mapper
    model = small_model

In [ ]:
validate = SFDataset(
    date_min="2016-01-01",
    date_max="2018-06-30",
    fields=conf.FEATURES,
    sample_size=1.7e6,
)
validate.data = pd.read_csv("/home/common/benchmark/052021_split_data_validation.csv")
    
test = conf.TEST_DATASET
test.sample_size = 1.2e6
#test.fetch_data()
test.data = pd.read_csv("/home/common/benchmark/052021_full_data_test.csv")

predict = conf.PREDICT_DATASET
predict.sample_size = 5e5
#predict.fetch_data()
predict.data = pd.read_csv("/home/simon.lebastard/predictsignauxfaibles/data/052021_features_feb2020.csv")

## Threshold selection for default model

In [ ]:
validate.replace_missing_data().remove_na(ignore=IGNORE_NA)
validate.data = run_pipeline(validate.data, conf.TRANSFO_PIPELINE)
validate.data["predicted_probability"] = pp.predict_proba(validate.data)[:, 1]

test.replace_missing_data().remove_na(ignore=IGNORE_NA)
test.data = run_pipeline(test.data, conf.TRANSFO_PIPELINE)
test.data["predicted_probability"] = pp.predict_proba(test.data)[:, 1]

predict.replace_missing_data().remove_na(ignore=IGNORE_NA)
predict.data = run_pipeline(predict.data, conf.TRANSFO_PIPELINE)
predict.data["predicted_probability"] = pp.predict_proba(predict.data)[:, 1]

Isolating transformed features on one hand, outcomes on the other:

In [ ]:
validate_features_unmapped = validate.data[set(validate.data.columns).difference(set(["outcome"]))]
validate_outcomes = validate.data["outcome"].astype(int).to_numpy()
validate_features = mapper.transform(validate_features_unmapped)

test_features_unmapped = test.data[set(test.data.columns).difference(set(["outcome"]))]
test_outcomes = test.data["outcome"].astype(int).to_numpy()
test_features = mapper.transform(test_features_unmapped)

In [ ]:
test.data.columns

In [ ]:
test.data.sample(n=3)

In [ ]:
100*test.data.groupby(by="outcome").siret.count()/len(test)

### Computing precision/recall graph as a function of classification thresholds - Default model

Let's load a test set from which we can use the output to measure our performance

In [ ]:
val_precision, val_recall, val_thresh = make_precision_recall_curve(
    validate,
    pp
)

test_precision, test_recall, test_thresh = make_precision_recall_curve(
    test,
    pp
)

### Computing AUCPR
Computing the Area Under Curve fopr Precision-Recall curve (AUCPR), a metric which summarizes the potential of the model itself, without specifying the hyperparameter tuning that must be led to weight the relative importance of  precision and recall.

In [ ]:
aucpr = average_precision_score(test_outcomes, pp.predict_proba(test.data)[:, 1])

In [ ]:
print(
    "Area under Precision-Recall curve: {0:.3f}".format(aucpr)
)

### Option 1: Determine the thresholds by hand, looking at the Type2-Type1 errors plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

type_2 = 1 - val_recall
type_1 = 1 - val_precision

ax.scatter(
    type_1,
    type_2,
    label = "logreg"
);

selected_F1_T1error = 0.08
selected_F1_precision = 1 - selected_F1_T1error

selected_F2_T2error = 0.415
selected_F2_recall = 1 - selected_F2_T2error

plt.axhline(y=selected_F2_T2error, color='orange', linestyle='-')
plt.axvline(x=selected_F1_T1error, color='red', linestyle='-')

ax.set_xlabel('Type 1 error', fontsize=16)
ax.set_ylabel('Type 2 error', fontsize=16)

ax.legend(fontsize=18);

On peut sélectionner un seuil F1 (rouge) et un seuil F2 (orange) à partir de cette courbe:
- pour la liste F1, on cherche à maximiser la précision, puisqe la liste en question doit être la plus pertinent possible et contenir des entreprises "prioritaires". Cette liste minimise donc les faux positifs, favorise la précision et minimise donc l'erreur de type 1.
- la liste F2 doit se charger de minimiser les faux négatifs, et est donc plus orienter sur la maximisation du recall. Il s'agit ici de minimiser l'erreur de type 2.

Pour $t_{F1}$, on choisit une valeur qui donne une erreur de type 1 petite tout en étant la plus indulgente possible pour l'erreur de type 2. C'est typiquement ce qu'un point de courbure maximale va réaliser. Ici $t_{F1}=0.07$ semble assez satisfaisant (ligne verticale rouge ci-dessus).

Pour $t_{F2}$, on choisir au contraire une valeur de courbe (négative) minimale, tout en satisfaisant déjà une erreur de type 2 suffisament faible. Ici $t_{F2}=0.37$ semble assez satisfaisant (ligne horizontale orange ci-dessus).

In [ ]:
(t_F1_cond, t_F2_cond) = make_thresholds_from_conditions(
    val_precision,
    val_recall,
    val_thresh,
    min_precision_F1 = selected_F1_precision,
    min_recall_F2 = selected_F2_recall,
)

#### DEFAULT model
Sur le dataset de test chargé depuis `/home/common/benchmark/052021_split_data_validation.csv` (1.7M):<br>
F1 - $t_{F1}=0.822$ garantit une précision de 92% pour la liste F1 (rouge)<br>
F2 - $t_{F2}=0.141$ garantit un recall de 58.5% pour la liste F2 (orange)

#### SMALL model
Pour le modèle "small“, sur le dataset de test chargé depuis `/home/common/benchmark/052021_full_data_test.csv`, la courbe précision/recall est trop "bruitée" pour une sélection de seuil semi-manuelle.

In [ ]:
log_splits_size(
    predict.data["predicted_probability"],
    t_F1=t_F1_cond,
    t_F2=t_F2_cond,
)

#### DEFAULT model
Avec les seuils sélectionnés juste au-dessus par une sélection semi-manuelle:
- Alerte seuil F1 ("risque fort", rouge): 1914  (1.7%)
- Alerte seuil F2 ("risque modéré", orange): 8496  (7.55%)

#### SMALL model
Pour le modèle "small“, sur le dataset de test chargé depuis `/home/common/benchmark/052021_full_data_test.csv`, la courbe précision/recall est trop "bruitée" pour une sélection de seuil semi-manuelle.

In [ ]:
evaluate(
    pp,
    test,
    beta=0.5,
    thresh=t_F1_cond
)

In [ ]:
evaluate(
    pp,
    test,
    beta=2,
    thresh=t_F2_cond
)

### Option 2: Select thresholds by maximising a f-beta (with beta specified for each alert level)

In [ ]:
validate_sample = validate.data.sample(n=int(1e5))

In [ ]:
(t_F1_fb, t_F2_fb) = make_thresholds_from_fbeta(
    validate_sample,
    validate_sample["outcome"].astype(int).to_numpy(), #validate_outcomes,
    pp,
    beta_F1 = 0.5,
    beta_F2 = 2,
    n_thr = 1000
)

#### DEFAULT model
Sur le dataset de test chargé depuis `/home/common/benchmark/052021_full_data_test.csv` (1.2M):<br>
F1 - $\beta=0.5$ - Optimal threshold: $t_{F1}=0.817$ - $f_{0.5}=0.722$<br>
F2 - $\beta=2$ - Optimal threshold: $t_{F2}=0.179$ - $f_{2}=0.547$

#### SMALL model
Sur le dataset de test chargé depuis `/home/common/benchmark/052021_full_data_test.csv` (1.2M):<br>
F1 - $\beta=0.5$ - Optimal threshold: $t_{F1}=0.667$ - $f_{0.5}=0.724$<br>
F2 - $\beta=2$ - Optimal threshold: $t_{F2}=0.134$ - $f_{2}=0.563$

In [ ]:
log_splits_size(
    predict.data["predicted_probability"],
    t_F1=t_F1_fb,
    t_F2=t_F2_fb,
)

#### DEFAULT model
Avec les seuils sélectionnés juste au-dessus par une maximisation des scores $f_{0.5}$ et $f_{2}$, respectivement:
- Alerte seuil F1 (risque fort, rouge): 1929 (1.71%)
- Alerte seuil F2 (risque modéré, orange): 5227 (4.65%)

#### SMALL model
Avec les seuils sélectionnés juste au-dessus par une maximisation des scores $f_{0.5}$ et $f_{2}$, respectivement:
- Alerte seuil F1 (risque fort, rouge): 1896 (1.69%)
- Alerte seuil F2 (risque modéré, orange): 3237 (2.88%)

In [ ]:
evaluate(
    pp,
    test,
    beta=0.5,
    thresh=t_F1_fb
)

In [ ]:
evaluate(
    pp,
    test,
    beta=2,
    thresh=t_F2_fb
)

## To go further - Can we build a threshold selection criterion based on the variations of recall % threhold?

In [ ]:
# Let's compute the first-order derivative of the type-2 error
fod_recall = (recall[:-1] - np.roll(recall[:-1],1))/(thresh - np.roll(thresh,1))
fod_recall = fod_recall[1:]

In [ ]:
plt.hist(fod_recall)

In [ ]:
plt.plot(thresh)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

ax.scatter(
    thresh[:-81],
    fod_recall[:-80],
    label = "logreg"
);

In [ ]:
recalL_maxvar_id = np.argmin(fod_recall[:-80])
recall_maxvar = fod_recall[recalL_maxvar_id]

recall_maxvar_precision = precision[recalL_maxvar_id]
recall_maxvar_recall = recall[recalL_maxvar_id]
recall_maxvar_thresh = thresh[recalL_maxvar_id]

In [ ]:
recall_maxvar_thresh

In [ ]:
recall_maxvar_recall

In [ ]:
recall_maxvar_precision

For now, we'll stick to manual selection based on the two types of error above, or maybe based on the maximisation of Fbeta